# Power System Analysis Summary
#
This notebook documents the final analytics workflow for the ENGR 010 electrical engineering project. It mirrors the production code, highlights the sequence of computations, and captures the reasoning behind each analytical step.

## 1. Environment Setup
We reuse the shared analysis modules so the notebook always reflects the same business logic as the batch script and dashboard.

In [ ]:
from pathlib import Path
import pandas as pd

from analysis_core import (
    GridStandards,
    calculate_basic_statistics,
    calculate_circuit_metrics,
    calculate_power_quality_indices,
    compare_to_standards,
    identify_load_patterns,
    load_power_data,
    perform_fault_analysis,
 )
from power_analysis_script import create_visualizations


## 2. Load the Source Dataset
We ingest the curated CSV once and hold onto the resulting DataFrame, matching what the batch pipeline and dashboard do.

In [ ]:
DATA_PATH = Path("power_system_data.csv")
standards = GridStandards()

dataset = load_power_data(DATA_PATH)
dataset.head()

### Dataset Scope Check
A quick glance at the time bounds and station coverage helps confirm we are aligned with expectations.

In [ ]:
summary = {
    "records": len(dataset),
    "time_range": (dataset['timestamp'].min(), dataset['timestamp'].max()),
    "stations": sorted(dataset['station_id'].unique()),
}
summary

## 3. Basic Statistics
The helper aggregates capture mean, median, and standard deviation for each electrical quantity, giving a quick read on central tendency and spread.

In [ ]:
basic_stats = calculate_basic_statistics(dataset)
basic_stats

## 4. Load Pattern Extraction
Daily, weekly, and hourly aggregations reveal seasonality and typical diurnal behavior.

In [ ]:
load_patterns = identify_load_patterns(dataset)
daily_sample = load_patterns["daily"].head()
weekly_sample = load_patterns["weekly"].head()
hourly_sample = load_patterns["hourly_profile"].head()

daily_sample, weekly_sample, hourly_sample

## 5. Power Quality and Circuit Health
These tables diagnose issues by comparing against grid standards and recommending power-factor correction.

In [ ]:
quality_indices = calculate_power_quality_indices(dataset, standards)
circuit_metrics = calculate_circuit_metrics(dataset, standards)
quality_indices, circuit_metrics

## 6. Standards Compliance and Fault Review
We reuse the same routines as the dashboard to quantify thresholds and isolated incidents.

In [ ]:
standards_comparison = compare_to_standards(dataset, standards)
fault_summary = perform_fault_analysis(dataset, standards)
standards_comparison, fault_summary

## 7. Regenerating Visual Outputs
The helper functions below mirror the batch script exports so we can refresh graphics within the notebook if needed.

In [ ]:
reports_dir = Path("analysis_outputs/reports")
figures_dir = Path("analysis_outputs/figures")

reports_dir.mkdir(parents=True, exist_ok=True)
figures_dir.mkdir(parents=True, exist_ok=True)

create_visualizations(dataset, load_patterns, circuit_metrics, figures_dir)
reports_dir, figures_dir

## 8. Next Steps
- The batch script (`power_analysis_script.py`) wraps the same workflow for automated exports.
- The Pygame dashboard (`main.py`) consumes these tables for live exploration.
- Future experiments can plug in alternate datasets or extend the analytics module without diverging from this notebook.